![erd](./images/ERD-week5-exercise.png)

## Instruction

1. ใช้ project myshop เดิมจากของ WEEK 4 มาทำต่อได้เลยครับ (ใครที่หา project ของ WEEK 4 ไม่เจอแล้ว ให้กลับไปทำตามขั้นตอนใน week4_exercises.ipynb)
2. ใช้ database `shop` เดิมได้เลยเช่นกันครับ

**หมายเหตุ: ถ้านักศึกษาใช้ database เดิมจาก WEEK4 ซึ่งนักศึกษาเคยเพิ่มหรือแก้ไขข้อมูลบางส่วนแล้ว ดังนั้น output อาจจะไม่ตรงกับในตัวอย่างทั้งหมดไม่ต้องตกใจนะครับ**

In [91]:
import os
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

# import modules
from shop.models import *
from django.db.models import F, Q
from django.db.models.functions import Concat
import json
from django.db.models import OuterRef, Subquery, Sum

### 1. annotate(), F()

1.1 ให้นักศึกษา query ข้อมูล `Payment` โดยให้เพิ่ม field ราคาที่ลบกับส่วนลดแล้ว โดยกำหนดให้ชื่อ field ว่า "after_discount_price" โดยใช้แสดงข้อมูล 10 ตัวแรกเรียงตาม "after_discount_price" จากมากไปน้อย (0.25 คะแนน)

**หมายเหตุ: จะต้องใช้ annotate() นะครับ ให้เอา `Payment.price` - `Payment.discount`**

ตัวอย่าง Output

```
ID: 92, PRICE: 1200500.00, DISCOUNT 29433.25, AFTER_DISCOUNT 1171066.75
ID: 82, PRICE: 1200280.00, DISCOUNT 46229.40, AFTER_DISCOUNT 1154050.60
ID: 137, PRICE: 1200690.00, DISCOUNT 71407.25, AFTER_DISCOUNT 1129282.75
ID: 105, PRICE: 1200390.00, DISCOUNT 105019.11, AFTER_DISCOUNT 1095370.89
ID: 45, PRICE: 1218900.00, DISCOUNT 126859.95, AFTER_DISCOUNT 1092040.05
ID: 7, PRICE: 1201200.00, DISCOUNT 113446.20, AFTER_DISCOUNT 1087753.80
ID: 18, PRICE: 1202190.00, DISCOUNT 121922.64, AFTER_DISCOUNT 1080267.36
ID: 77, PRICE: 379000.00, DISCOUNT 19397.00, AFTER_DISCOUNT 359603.00
ID: 127, PRICE: 320450.00, DISCOUNT 14578.90, AFTER_DISCOUNT 305871.10
ID: 125, PRICE: 320399.00, DISCOUNT 17939.55, AFTER_DISCOUNT 302459.45
```

In [23]:
payments = Payment.objects.annotate(after_discount_price=F("price") - F("discount")).order_by("-after_discount_price")[:10]

for payment in payments:
    print(f"ID: {payment.id}, PRICE: {payment.price}, DISCOUNT {payment.discount}, AFTER_DISCOUNT {payment.after_discount_price}")

ID: 92, PRICE: 1200500.00, DISCOUNT 29433.25, AFTER_DISCOUNT 1171066.75
ID: 82, PRICE: 1200280.00, DISCOUNT 46229.40, AFTER_DISCOUNT 1154050.60
ID: 137, PRICE: 1200690.00, DISCOUNT 71407.25, AFTER_DISCOUNT 1129282.75
ID: 105, PRICE: 1200390.00, DISCOUNT 105019.11, AFTER_DISCOUNT 1095370.89
ID: 45, PRICE: 1218900.00, DISCOUNT 126859.95, AFTER_DISCOUNT 1092040.05
ID: 7, PRICE: 1201200.00, DISCOUNT 113446.20, AFTER_DISCOUNT 1087753.80
ID: 18, PRICE: 1202190.00, DISCOUNT 121922.64, AFTER_DISCOUNT 1080267.36
ID: 77, PRICE: 379000.00, DISCOUNT 19397.00, AFTER_DISCOUNT 359603.00
ID: 127, PRICE: 320450.00, DISCOUNT 14578.90, AFTER_DISCOUNT 305871.10
ID: 125, PRICE: 320399.00, DISCOUNT 17939.55, AFTER_DISCOUNT 302459.45


1.2 ต่อเนื่องจากข้อ 1.1 ให้ filter เฉพาะข้อมูล `Payment` ที่มี "after_discount_price" มากกว่า 500,000 (0.25 คะแนน)

ตัวอย่าง Output

```
ID: 92, PRICE: 1200500.00, DISCOUNT 29433.25, AFTER_DISCOUNT 1171066.75
ID: 82, PRICE: 1200280.00, DISCOUNT 46229.40, AFTER_DISCOUNT 1154050.60
ID: 137, PRICE: 1200690.00, DISCOUNT 71407.25, AFTER_DISCOUNT 1129282.75
ID: 105, PRICE: 1200390.00, DISCOUNT 105019.11, AFTER_DISCOUNT 1095370.89
ID: 45, PRICE: 1218900.00, DISCOUNT 126859.95, AFTER_DISCOUNT 1092040.05
ID: 7, PRICE: 1201200.00, DISCOUNT 113446.20, AFTER_DISCOUNT 1087753.80
ID: 18, PRICE: 1202190.00, DISCOUNT 121922.64, AFTER_DISCOUNT 1080267.36
```

In [25]:
payments = Payment.objects.annotate(after_discount_price=F("price") - F("discount")).filter(after_discount_price__gt=500000).order_by("-after_discount_price")[:10]

for payment in payments:
    print(f"ID: {payment.id}, PRICE: {payment.price}, DISCOUNT {payment.discount}, AFTER_DISCOUNT {payment.after_discount_price}")

ID: 92, PRICE: 1200500.00, DISCOUNT 29433.25, AFTER_DISCOUNT 1171066.75
ID: 82, PRICE: 1200280.00, DISCOUNT 46229.40, AFTER_DISCOUNT 1154050.60
ID: 137, PRICE: 1200690.00, DISCOUNT 71407.25, AFTER_DISCOUNT 1129282.75
ID: 105, PRICE: 1200390.00, DISCOUNT 105019.11, AFTER_DISCOUNT 1095370.89
ID: 45, PRICE: 1218900.00, DISCOUNT 126859.95, AFTER_DISCOUNT 1092040.05
ID: 7, PRICE: 1201200.00, DISCOUNT 113446.20, AFTER_DISCOUNT 1087753.80
ID: 18, PRICE: 1202190.00, DISCOUNT 121922.64, AFTER_DISCOUNT 1080267.36


1.3 ให้นักศึกษาเรียงลำดับข้อมูลลูกค้า (`Customer`) โดยเรียงลำดับตามลำดับตัวอักษร `น้อยไปมาก` (A-Z) จากชื่อเต็มของลูกค้า (`full_name`) โดยแสดง 5 คนแรก (0.5 คะแนน)

**Hint:** Field `full_name` นั้นจะต้องถูก annotate ขึ้นมาโดยการนำ `first_name` มาต่อกับ `last_name` โดยใช้ `Concat(*expressions, **extra)` 

**Hint:** แปลง object เป็น dict ใช้ `values()` [doc](https://docs.djangoproject.com/en/5.0/ref/models/querysets/#values)

```python
>>> Blog.objects.filter(name__startswith="Beatles").values()
<QuerySet [{'id': 1, 'name': 'Beatles Blog', 'tagline': 'All the latest Beatles news.'}]>
```

**Hint:** อยาก print dictionary สวยๆ ใช้ `json.dumps`

```python
print(json.dumps(dictionary, indent=4, sort_keys=False))
```

[doc](https://docs.djangoproject.com/en/5.0/ref/models/database-functions/#concat)

ตัวอย่าง Output 

```python
{
    "id": 17,
    "email": "anantaya.deena@gmail.com",
    "address": {
        "district": "Yan Nawa",
        "location": "60 Thanon Chan Road",
        "province": "Bangkok",
        "postal_code": 10120
    },
    "full_name": "Anantaya Tontong"
}
{
    "id": 25,
    "email": "bancha.zeyou@gmail.com",
    "address": {
        "district": "Watthana",
        "location": "6 Thong Lo Road",
        "province": "Bangkok",
        "postal_code": 10110
    },
    "full_name": "Bancha Kittisompong"
}
{
    "id": 19,
    "email": "chayapol.231@gmail.com",
    "address": {
        "district": "Hang Chat",
        "location": "160 Lampang Road",
        "province": "Lampang",
        "postal_code": 52190
    },
    "full_name": "Chayapol Komprach"
}
{
    "id": 4,
    "email": "dejwit.tt@gmail.com",
    "address": {
        "district": "Chiang Khan",
        "location": "150 Loei Road",
        "province": "Loei",
        "postal_code": 42110
    },
    "full_name": "Dejwit Tangjareonsakul"
}
{
    "id": 11,
    "email": "jack.maa@gmail.com",
    "address": {
        "district": "Bang Khen",
        "location": "88 Phahonyothin Road",
        "province": "Bangkok",
        "postal_code": 10220
    },
    "full_name": "Jack Maa"
}

```

In [51]:
customers = Customer.objects.annotate(full_name=Concat("first_name", "last_name")).order_by("full_name").values()[:5]

for customer in customers:
    print(json.dumps(customer, indent=4, sort_keys=False))

{
    "id": 17,
    "first_name": "Anantaya",
    "last_name": "Tontong",
    "email": "anantaya.deena@gmail.com",
    "address": {
        "district": "Yan Nawa",
        "location": "60 Thanon Chan Road",
        "province": "Bangkok",
        "postal_code": 10120
    },
    "full_name": "AnantayaTontong"
}
{
    "id": 25,
    "first_name": "Bancha",
    "last_name": "Kittisompong",
    "email": "bancha.zeyou@gmail.com",
    "address": {
        "district": "Watthana",
        "location": "6 Thong Lo Road",
        "province": "Bangkok",
        "postal_code": 10110
    },
    "full_name": "BanchaKittisompong"
}
{
    "id": 19,
    "first_name": "Chayapol",
    "last_name": "Komprach",
    "email": "chayapol.231@gmail.com",
    "address": {
        "district": "Hang Chat",
        "location": "160 Lampang Road",
        "province": "Lampang",
        "postal_code": 52190
    },
    "full_name": "ChayapolKomprach"
}
{
    "id": 4,
    "first_name": "Dejwit",
    "last_name": "Tangjare

### 3. aggregation - count(), sum(), avg()


3.1 ให้นักศึกษาหาค่าเฉลี่ยของราคาสินค้า (`Product.price`) ที่มีจำนวนคงเหลือ (`Product.remaining_amount`) มากกว่า 0 (0.2 คะแนน)

``` PYTHON
{'avg': Decimal('29308.000000000000')}
```


In [57]:
Product.objects.filter(remaining_amount__gt=0).aggregate(Avg("price"))

{'price__avg': Decimal('29308.000000000000')}

3.2 จงหาผลรวมราคา (`CartItem.product.price`) ที่อยู่ในตระกร้าสินค้าของเดือน `พฤษภาคม` (ดูจาก `Cart.create_date`) (0.2 คะแนน)

```PYTHON
{'sum': Decimal('9912555.00')}

```

In [61]:
CartItem.objects.filter(cart__create_date__month="05").aggregate(Sum("product__price"))

{'product__price__sum': Decimal('9912555.00')}

3.3 ให้นักศึกษานับจำนวนสินค้าที่อยู่ประเภท `Electronics`,  `Jewelry` และ ราคาของสินค้าอยู่ในช่วง 8,000.00 - 50,000.00 (0.2 คะแนน)

```
PRODUCT CATEGORY NAME: Electronics, PRODUCT COUNT: 6
PRODUCT CATEGORY NAME: Jewelry, PRODUCT COUNT: 1
```

In [90]:
productCats = Product.objects.filter(Q(categories__name="Electronics") | Q(categories__name="Jewelry"), price__gte=8000, price__lte=50000).values("categories__name").annotate(count=Count("categories"))

for cate in productCats:
    print(f"PRODUCT CATEGORY NAME: {cate["categories__name"]}, PRODUCT COUNT: {cate["count"]}")

PRODUCT CATEGORY NAME: Electronics, PRODUCT COUNT: 6
PRODUCT CATEGORY NAME: Jewelry, PRODUCT COUNT: 1


In [120]:
orders = Order.objects.filter(customer=OuterRef('pk')).values('order_date')[:1]
customers = Customer.objects.annotate(last_order=Subquery(orders)).order_by("last_order")

for customer in customers:
    print(f"CUSTOMER NAME: {customer.first_name} {customer.last_name}, ORDER DATE: {customer.last_order}")

CUSTOMER NAME: Pakin Janpen, ORDER DATE: 2024-04-24
CUSTOMER NAME: Jenjira Sukanansarn, ORDER DATE: 2024-04-24
CUSTOMER NAME: Pong Sawadiwong, ORDER DATE: 2024-04-24
CUSTOMER NAME: Kwantip Aekopas, ORDER DATE: 2024-04-25
CUSTOMER NAME: Jakkapob Sopapak, ORDER DATE: 2024-04-25
CUSTOMER NAME: Dejwit Tangjareonsakul, ORDER DATE: 2024-04-26
CUSTOMER NAME: Suwichada Jarastham, ORDER DATE: 2024-04-26
CUSTOMER NAME: Anantaya Tontong, ORDER DATE: 2024-04-26
CUSTOMER NAME: Techathorn Sriwanwilai, ORDER DATE: 2024-04-27
CUSTOMER NAME: Trinnapon Yinggnam, ORDER DATE: 2024-04-27
CUSTOMER NAME: Chayapol Komprach, ORDER DATE: 2024-04-28
CUSTOMER NAME: Yanisa Thongchaiyapoom, ORDER DATE: 2024-04-28
CUSTOMER NAME: Pattanapoom Srisawangjan, ORDER DATE: 2024-04-29
CUSTOMER NAME: Phuwadech Siriwanich, ORDER DATE: 2024-04-29
CUSTOMER NAME: Kerikwan Laopong, ORDER DATE: 2024-04-30
CUSTOMER NAME: Warit Pititat, ORDER DATE: 2024-05-01
CUSTOMER NAME: Rachanon Hongpatra, ORDER DATE: 2024-05-01
CUSTOMER NAME: P

### 4. one-to-one & one-to-many

4.1 จงสร้าง Order และ Payment ของลูกค้า Sek Loso จำนวน 2 รายการ โดยแต่ละ Order มีสินค้า 1 รายการ และชำระเงินด้วย Payment คนละรายการ จากนั้น เขียน query แสดงข้อมูล Order และ Payment ทั้งหมดของ Sek Loso (0.25 คะแนน)
```txt
# ตัวอย่างผลลัพธ์บางส่วน
CUSTOMER: Sek Loso, ORDER ID: 48, PRICE: 1699.00, DISCOUNT: 112.27, PRODUCT: Bird Cage
CUSTOMER: Sek Loso, ORDER ID: 59, PRICE: 1740.00, DISCOUNT: 140.81, PRODUCT: Microwave Oven
```

In [175]:
pSek = Customer.objects.get(first_name="Sek", last_name="Loso")
birdCage = Product.objects.get(name="Bird Cage")
microwaveOven = Product.objects.get(name="Microwave Oven")

newOrder = Order.objects.create(customer=pSek, order_date="2025-01-01")
newOrder2 = Order.objects.create(customer=pSek, order_date="2025-01-02")

newOrderItem = OrderItem.objects.create(order=newOrder, product=birdCage)
newOrderItem2 = OrderItem.objects.create(order=newOrder2, product=microwaveOven)

newPayment = Payment.objects.create(order=newOrder, payment_date="2025-05-01", price=1739.00, discount=234.00)
newPayment2 = Payment.objects.create(order=newOrder2, payment_date="2025-05-02", price=1839.00, discount=334.00)

newPaymentItem = PaymentItem.objects.create(payment=newPayment, order_item=newOrderItem, price=100.00, discount=99.00)
newPaymentItem2 = PaymentItem.objects.create(payment=newPayment2, order_item=newOrderItem2, price=140.00, discount=69.00)

newPaymentMethod = PaymentMethod.objects.create(payment=newPayment, method="QR", price=1892.00)
newPaymentMethod2 = PaymentMethod.objects.create(payment=newPayment2, method="CREDIT", price=1929.00)



orders = Order.objects.filter(customer=pSek)

for order in orders:
    orderItems = OrderItem.objects.filter(order=order)
    products = [orderItem.product.name for orderItem in orderItems]
    payment = Payment.objects.get(order=order)
    print(f"CUSTOMER: {pSek.first_name} {pSek.last_name}, ORDER ID: {order.id}, PRICE: {payment.price}, DISCOUNT: {payment.discount}, PRODUCT: {", ".join(products)}")

CUSTOMER: Sek Loso, ORDER ID: 48, PRICE: 1699.00, DISCOUNT: 112.27, PRODUCT: Bird Cage, Hot Wheels Track Builder System Race Crate
CUSTOMER: Sek Loso, ORDER ID: 59, PRICE: 1740.00, DISCOUNT: 140.81, PRODUCT: Microwave Oven, Men's Polo Shirt
CUSTOMER: Sek Loso, ORDER ID: 184, PRICE: 1739.00, DISCOUNT: 234.00, PRODUCT: Bird Cage
CUSTOMER: Sek Loso, ORDER ID: 185, PRICE: 1839.00, DISCOUNT: 334.00, PRODUCT: Microwave Oven


4.2 ให้นักศึกษาดึงข้อมูล `Payment` ที่มี `PaymentMethod` ที่เป็น 'CREDIT' และมีราคาหลังหักส่วนลด (`Payment.price` - `Payment.discount`) ต่ำกว่า 500 และให้แสดงจำนวนของ `PaymentItem` ของแต่ละรายการ Payment (0.25 คะแนน)
```txt
# ตัวอย่างผลลัพธ์
PAYMENT ID: 40, AFTER_DISCOUNT: 493.06, ITEM COUNT: 2
PAYMENT ID: 13, AFTER_DISCOUNT: 448.54, ITEM COUNT: 2
```

In [186]:
payments = Payment.objects.annotate(
    disc_price=F("price")-F("discount"),
    count=Count("paymentitem")
).filter(paymentmethod__method="CREDIT", disc_price__lt=500)

for payment in payments:
    
    print(f"PAYMENT ID: {payment.id}, AFTER_DISCOUNT: {payment.disc_price}, ITEM COUNT: {payment.count}")

PAYMENT ID: 40, AFTER_DISCOUNT: 493.06, ITEM COUNT: 2
PAYMENT ID: 13, AFTER_DISCOUNT: 448.54, ITEM COUNT: 2


### 5. many-to-many

5.1 ให้นักศึกษาค้นหาข้อมูลสินค้า (`Product`) ที่อยู่ในทั้งหมวดหมู่สินค้า "Information Technology" และ "Electronics" 10 รายการแรก (เรียงลำดับด้วย `Product.id`) และแสดงชื่อประเภทสินค้า (`ProductCategory`) (0.25 คะแนน)

ตัวอย่าง Output บางส่วน

```
PRODUCT ID: 1, PRODUCT NAME: Smartphone, PRODUCT CATEGORY: Information technology, Electronics
PRODUCT ID: 2, PRODUCT NAME: Laptop, PRODUCT CATEGORY: Information technology, Electronics
PRODUCT ID: 3, PRODUCT NAME: Smart TV, PRODUCT CATEGORY: Information technology, Electronics
```

In [225]:
it = ProductCategory.objects.get(name="Information Technology")
elec = ProductCategory.objects.get(name="Electronics")

products = Product.objects.filter(Q(categories=it) | Q(categories=elec)).order_by("id").distinct()[:10]

for product in products:
    productCat = [cat.name for cat in product.categories.all()]
    print(f"PRODUCT ID: {product.id}, PRODUCT NAME: {product.name}, PRODUCT CATEGORY: {", ".join(productCat)}")

DoesNotExist: ProductCategory matching query does not exist.

5.2 ให้นักศึกษาทำตามขั้นตอนดังนี้ (0.25 คะแนน)

1. เปลี่ยนชื่อหมวดหมู่สินค้า `Electronics` เป็น `Electronics and Toys` แทน
2. ย้ายสินค้าที่อยู่ใน `Toys and Games` ไปอยู่ในหมวดหมู่ `Electronics and Toys` และ ลบหมวดหมู่สินค้า `Toys and Games` 
3. ค้นหาว่าสินค้าที่มีประเภทสินค้าเป็น `Electronics and Toys` ทั้งหมดมีจำนวนเท่าไหร่

In [223]:
elec = ProductCategory.objects.get(name="Electronics and Toys")
elec.name = "Electronics and Toys"
elec.save()

toygame = ProductCategory.objects.get(name="Toys and Games")
toyProducts = Product.objects.filter(categories=toygame)

for product in toyProducts:
    product.categories.remove(toygame)
    product.categories.add(elec)

toygame.delete()

allElec = Product.objects.filter(categories=elec).count()
print(allElec)

<bound method QuerySet.count of <QuerySet [<ProductCategory: ProductCategory object (2)>]>>
